In [1]:
import numpy as np
import os
import cv2 as cv
import pickle
from skimage.feature import hog
import copy
import time

In [2]:
def IOU(bbox_a, bbox_b):
    x_a = max(bbox_a[0], bbox_b[0])
    y_a = max(bbox_a[1], bbox_b[1])
    x_b = min(bbox_a[2], bbox_b[2])
    y_b = min(bbox_a[3], bbox_b[3])

    inter_area = max(0, x_b - x_a + 1) * max(0, y_b - y_a + 1)

    box_a_area = (bbox_a[2] - bbox_a[0] + 1) * (bbox_a[3] - bbox_a[1] + 1)
    box_b_area = (bbox_b[2] - bbox_b[0] + 1) * (bbox_b[3] - bbox_b[1] + 1)

    iou = inter_area / float(box_a_area + box_b_area - inter_area)

    return iou


In [3]:
def non_maximal_suppression(image_detections, image_scores, image_size):
    """
    Detectiile cu scor mare suprima detectiile ce se suprapun cu acestea dar au scor mai mic.
    Detectiile se pot suprapune partial, dar centrul unei detectii nu poate
    fi in interiorul celeilalte detectii.
    :param image_detections:  numpy array de dimensiune NX4, unde N este numarul de detectii.
    :param image_scores: numpy array de dimensiune N
    :param image_size: tuplu, dimensiunea imaginii
    :return: image_detections si image_scores care sunt maximale.
    """

    # xmin, ymin, xmax, ymax
    # print(image_size)
    x_out_of_bounds = np.where(image_detections[:, 2] > image_size[1])[0]
    y_out_of_bounds = np.where(image_detections[:, 3] > image_size[0])[0]
    # print(x_out_of_bounds, y_out_of_bounds)
    image_detections[x_out_of_bounds, 2] = image_size[1]
    image_detections[y_out_of_bounds, 3] = image_size[0]
    sorted_indices = np.flipud(np.argsort(image_scores))
    sorted_image_detections = image_detections[sorted_indices]
    sorted_scores = image_scores[sorted_indices]

    is_maximal = np.ones(len(image_detections)).astype(bool)
    iou_threshold = 0.3
    for i in range(len(sorted_image_detections) - 1):
        if is_maximal[i] == True:  # don't change to 'is True' because is a numpy True and is not a python True :)
            for j in range(i + 1, len(sorted_image_detections)):
                if is_maximal[j] == True:  # don't change to 'is True' because is a numpy True and is not a python True :)
                    if IOU(sorted_image_detections[i],sorted_image_detections[j]) > iou_threshold:
                        is_maximal[j] = False
                    else:  # verificam daca centrul detectiei este in mijlocul detectiei cu scor mai mare
                        c_x = (sorted_image_detections[j][0] + sorted_image_detections[j][2]) / 2
                        c_y = (sorted_image_detections[j][1] + sorted_image_detections[j][3]) / 2
                        if sorted_image_detections[i][0] <= c_x <= sorted_image_detections[i][2] and \
                                sorted_image_detections[i][1] <= c_y <= sorted_image_detections[i][3]:
                            is_maximal[j] = False
    return sorted_image_detections[is_maximal], sorted_scores[is_maximal]

In [4]:
lungime_fereastra_patrat,latime_fereastra_dreptunghi, inaltime_fereastra_dreptunghi,dim_cel_hog,orient =96,96,64,12,12

SVCpatrat_dexter = pickle.load(open(f'SVC/personaje/dexter__best_model_l{lungime_fereastra_patrat}_h{lungime_fereastra_patrat}_dimcelula{dim_cel_hog}.pkl', 'rb'))
SVCdreptunghi_dexter= pickle.load(open(f'SVC/personaje/dexter__best_model_l{latime_fereastra_dreptunghi}_h{inaltime_fereastra_dreptunghi}_dimcelula{dim_cel_hog}.pkl', 'rb'))
SVCdreptunghi_lung_dexter= pickle.load(open(f'SVC/personaje/dexter__best_model_l{inaltime_fereastra_dreptunghi}_h{latime_fereastra_dreptunghi}_dimcelula{dim_cel_hog}.pkl', 'rb'))


threshhold_detectie = 0.5
# print(SVCpatrat.coef_.T)
# print(SVCpatrat.intercept_[0])

In [5]:
def sliding_window_img(img,SVCpatrat, SVCdreptunghi,SVCdreptunghi_lung):
    scores = []
    detections = []
    img_hog = hog(img, pixels_per_cell=(dim_cel_hog, dim_cel_hog), cells_per_block=(2, 2), feature_vector=False,orientations = orient,transform_sqrt=True)
    # print(img_hog.shape)
    # print(SVCpatrat.coef_.T.shape)

    # pt fereastra patrata
    if lungime_fereastra_patrat <= img.shape[0] and lungime_fereastra_patrat <= img.shape[1]:
        # weight_patrat = SVCpatrat.coef_.T
        # bias_patrat = SVCpatrat.intercept_[0]
        num_cols = img.shape[1] // dim_cel_hog - 1
        num_rows = img.shape[0] // dim_cel_hog - 1
        num_cell_in_template = lungime_fereastra_patrat// dim_cel_hog - 1
        for y in range(0, num_rows - num_cell_in_template):
            for x in range(0, num_cols - num_cell_in_template):
                descr = img_hog[y:y + num_cell_in_template, x:x + num_cell_in_template].flatten()
            
                score = SVCpatrat.decision_function(descr.reshape(1, -1))[0]
                
                if score > threshhold_detectie:
                    x_min = int(x * dim_cel_hog)
                    y_min = int(y * dim_cel_hog)
                    x_max = int(x * dim_cel_hog + lungime_fereastra_patrat)
                    y_max = int(y * dim_cel_hog+ lungime_fereastra_patrat)
                    
                    detections.append([x_min, y_min, x_max, y_max])
                    scores.append(score)
        #pt fereastra dreptunghi
    if img.shape[0]>=latime_fereastra_dreptunghi and img.shape[1]>=inaltime_fereastra_dreptunghi:
        # weight_dreptunghi = SVCdreptunghi.coef_.T
        # bias_dreptunghi = SVCdreptunghi.intercept_[0]
        num_cols = img.shape[1] // dim_cel_hog - 1
        num_rows = img.shape[0] // dim_cel_hog - 1
        num_cell_in_template_latime = latime_fereastra_dreptunghi// dim_cel_hog - 1
        num_cell_in_template_inaltime = inaltime_fereastra_dreptunghi// dim_cel_hog - 1
        for y in range(0, num_rows - num_cell_in_template_inaltime):
            for x in range(0, num_cols - num_cell_in_template_latime):
                descr = img_hog[y:y + num_cell_in_template_inaltime, x:x + num_cell_in_template_latime].flatten()
                score = SVCdreptunghi.decision_function(descr.reshape(1, -1))[0]
                
                if score > threshhold_detectie:
                    x_min = int(x * dim_cel_hog)
                    y_min = int(y * dim_cel_hog)
                    x_max = int(x * dim_cel_hog + latime_fereastra_dreptunghi)
                    y_max = int(y * dim_cel_hog+ inaltime_fereastra_dreptunghi)
                    
                    detections.append([x_min, y_min, x_max, y_max])
                    scores.append(score)
    #pt fereastra dreptunghi lung
    if img.shape[0]>=inaltime_fereastra_dreptunghi and img.shape[1]>=latime_fereastra_dreptunghi:
        # weight_dreptunghi = SVCdreptunghi_lung.coef_.T
        # bias_dreptunghi = SVCdreptunghi_lung.intercept_[0]
        num_cols = img.shape[1] // dim_cel_hog - 1
        num_rows = img.shape[0] // dim_cel_hog - 1
        num_cell_in_template_latime = inaltime_fereastra_dreptunghi// dim_cel_hog - 1
        num_cell_in_template_inaltime = latime_fereastra_dreptunghi// dim_cel_hog - 1
        for y in range(0, num_rows - num_cell_in_template_inaltime):
            for x in range(0, num_cols - num_cell_in_template_latime):
                descr = img_hog[y:y + num_cell_in_template_inaltime, x:x + num_cell_in_template_latime].flatten()
                # score = np.dot(descr, weight_dreptunghi)[0] + bias_dreptunghi
                score = SVCdreptunghi_lung.decision_function(descr.reshape(1, -1))[0]
                
                if score > threshhold_detectie:
                    x_min = int(x * dim_cel_hog)
                    y_min = int(y * dim_cel_hog)
                    x_max = int(x * dim_cel_hog + inaltime_fereastra_dreptunghi)
                    y_max = int(y * dim_cel_hog+ latime_fereastra_dreptunghi)
                    

                    detections.append([x_min, y_min, x_max, y_max])
                    scores.append(score)
   
    scores = np.array(scores)
    detections = np.array(detections)
    return  detections, scores

In [6]:
def multiple_size_sliding_window(scala_initiala,step, fisier_imagini,SVCpatrat, SVCdreptunghi,SVCdreptunghi_lung):
    files = os.listdir(fisier_imagini)
    scoruri_totale = None
    detectii_totale = None
    file_names =[]
    for file in files:
        print(fisier_imagini + file)
        start_time = time.time()
        img_color = cv.imread(fisier_imagini + file)
        img = cv.cvtColor(img_color,cv.COLOR_BGR2GRAY)
        scoruri_per_img= None
        detectii_per_img= None
        scala_curr = scala_initiala
        img_init= copy.deepcopy(img)
        img = cv.resize(img, None, fx=scala_curr, fy=scala_curr)
        while (img.shape[0]>=lungime_fereastra_patrat and img.shape[1]>=lungime_fereastra_patrat ) or (img.shape[0]>=latime_fereastra_dreptunghi and img.shape[1]>=inaltime_fereastra_dreptunghi) or (img.shape[1]>=latime_fereastra_dreptunghi and img.shape[0]>=inaltime_fereastra_dreptunghi):
            # print("while")
            detection_curr, scores_curr = sliding_window_img(img,SVCpatrat, SVCdreptunghi,SVCdreptunghi_lung)
            # print(detection_curr.shape)
            if len(scores_curr)>0:
                detection_curr= detection_curr//scala_curr
                if scoruri_per_img is None :
                    scoruri_per_img = copy.deepcopy(scores_curr)
                    detectii_per_img = copy.deepcopy(detection_curr)
                else:
                    scoruri_per_img=np.concatenate((scoruri_per_img,scores_curr))
                    detectii_per_img=np.concatenate((detectii_per_img,detection_curr))

            scala_curr = scala_curr*step

            img = cv.resize(img_init, None, fx=scala_curr, fy=scala_curr)
        if scoruri_per_img is not None and len(scoruri_per_img) > 0:
            detectii_per_img,scoruri_per_img = non_maximal_suppression(detectii_per_img,scoruri_per_img,img_init.shape)
            
        if scoruri_per_img is not None and len(scoruri_per_img) >0:
            img_names = [file for _ in scoruri_per_img]
            file_names= np.concatenate((file_names,img_names))
            if scoruri_totale is None:
                scoruri_totale = copy.deepcopy(scoruri_per_img)
                detectii_totale = copy.deepcopy(detectii_per_img)
            else:
                scoruri_totale = np.concatenate((scoruri_totale,scoruri_per_img))
                detectii_totale = np.concatenate((detectii_totale,detectii_per_img))
        end_time = time.time()  # End timing
        print(f"Time taken for {file}: {end_time - start_time:.2f} seconds")
    return detectii_totale, scoruri_totale, file_names     

In [7]:
os.makedirs("./351_Pirvulescu_Daria",exist_ok=True)
os.makedirs("./351_Pirvulescu_Daria/task1",exist_ok=True)
os.makedirs("./351_Pirvulescu_Daria/task2",exist_ok=True)

In [8]:
############################
#aici se schimba
test_img_path='./testare/'
############################

detectii_totale, scoruri_finale, file_name = multiple_size_sliding_window(1,0.9,test_img_path,SVCpatrat_dexter, SVCdreptunghi_dexter,SVCdreptunghi_lung_dexter)
# os.makedirs("./detectii",exist_ok=True)
np.save("./351_Pirvulescu_Daria/task2/detections_dexter.npy", detectii_totale.astype(int))
np.save("./351_Pirvulescu_Daria/task2/scores_dexter.npy", scoruri_finale)
np.save("./351_Pirvulescu_Daria/task2/file_names_dexter.npy", file_name)
# print(detectii_totale)
# print(detectii_totale)


./testare/001.jpg
Time taken for 001.jpg: 73.65 seconds
./testare/002.jpg
Time taken for 002.jpg: 67.71 seconds
./testare/003.jpg
Time taken for 003.jpg: 66.54 seconds
./testare/004.jpg
Time taken for 004.jpg: 66.06 seconds
./testare/005.jpg
Time taken for 005.jpg: 67.99 seconds
./testare/006.jpg
Time taken for 006.jpg: 77.35 seconds
./testare/007.jpg
Time taken for 007.jpg: 78.09 seconds
./testare/008.jpg
Time taken for 008.jpg: 73.44 seconds
./testare/009.jpg
Time taken for 009.jpg: 66.21 seconds
./testare/010.jpg
Time taken for 010.jpg: 72.13 seconds
./testare/011.jpg
Time taken for 011.jpg: 78.56 seconds
./testare/012.jpg
Time taken for 012.jpg: 74.72 seconds
./testare/013.jpg
Time taken for 013.jpg: 74.75 seconds
./testare/014.jpg
Time taken for 014.jpg: 66.48 seconds
./testare/015.jpg
Time taken for 015.jpg: 65.95 seconds
./testare/016.jpg
Time taken for 016.jpg: 66.40 seconds
./testare/017.jpg
Time taken for 017.jpg: 65.58 seconds
./testare/018.jpg
Time taken for 018.jpg: 66.98 

In [9]:
print("hi")

hi


In [10]:
files= os.listdir("./testare/")
os.makedirs("./detected_faces_dexter/",exist_ok=True)
for file in files:
    img = cv.imread("./testare/"+ file)
    for det, score , name in zip(detectii_totale, scoruri_finale, file_name):
        if name== file:
            x_min, y_min, x_max, y_max = det
            score = (score - min(scoruri_finale)) / (max(scoruri_finale) - min(scoruri_finale) + 1e-6)
            green_intensity = int(min(max(score * 255, 0), 255))
            cv.rectangle(img, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0,green_intensity,0), 2)
    cv.imwrite(f"./detected_faces_dexter/{file}", img)